In [1]:

import sympy as sp
import numpy as np
from scipy.stats import ortho_group

# Skew-symmetric matrix function for cross product
def skew(vec_):
    """Returns the skew-symmetric matrix [vec]_x for a 3x1 vector."""
    vec = vec_.platten()
    return sp.Matrix([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

def skew_np(vec_):
    vec = np.squeeze(vec_.reshape(3,))

    """Returns the skew-symmetric matrix [vec]_x for a 3x1 vector."""
    return np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

# Generate a random 3x3 orthogonal matrix
random_orthogonal_matrix = ortho_group.rvs(3)
random_orthogonal_matrix_2 = ortho_group.rvs(3)
v_init = np.array([0, 0, 0]).reshape(3, 1)
p_init = np.array([0, 0, 0]).reshape(3, 1)
print(random_orthogonal_matrix_2)

[[ 0.82741152  0.11751103  0.54916422]
 [-0.25692946 -0.7903187   0.55622262]
 [-0.49937705  0.60132147  0.62372675]]


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.linalg import expm, logm


def h_acc(state, a_n, omega_n, i):
    R_imu, v_imu, p_imu, b_omega, b_a, R_c, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12], state[12:15], state[15:18], state[18:21]
    R_c = expm(skew_np(state[15:18].reshape(3,))) @ random_orthogonal_matrix
    previous_chi = np.vstack((
                            np.hstack((random_orthogonal_matrix_2, v_init, p_init)),
                            np.hstack((np.zeros((2, 3)), np.eye(2)))
                        ))
    
    skew_5_5 = np.vstack((
        np.hstack((skew_np(state[0:3].reshape(3,)), v_imu, p_imu)),
        np.zeros((2, 5))
    ))
    
    R_imu = (expm(skew_5_5) @ previous_chi )[:3, :3]

    b_vector = (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)).reshape(3,)
    
    v_c = R_c.T @ (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c))

    if(i == 0):
        print("the term 2/7 is \n", (R_c.T  @ R_imu.T).T)
        print("the term 4/7 is \n", (R_c.T @ skew_np(p_c.reshape(3,))).T)
        print("the term 6/7 B \n", (R_c.T @ skew_np (b_vector)).T)
        print("FOR term 7/7  \n", (R_c.T @ skew_np(omega_n - b_omega)).T)

    obs_paper = v_c
    return obs_paper

def compute_jacobian_paper(x_0, a_n, omega_n, d=1e-9):
    # Define small perturbations
    n = len(x_0)
    jacobian = np.zeros((3, n))

    # Compute the observation at the nominal state
    obs_nominal = h_acc(x_0, a_n, omega_n,10)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc(x_0_perturbed, a_n, omega_n, i)
        # Calculate Jacobian element by finite difference
        jacobian[:, i] = (obs_perturbed - obs_nominal) / d

    return jacobian

def h_acc_new(state, a_n, omega_n, i):
    R_imu1, v_imu, p_imu, b_omega, b_a, R_c_old, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12], state[12:15], state[15:18], state[18:21]
    
    R_c = expm(skew_np(state[15:18].reshape(3,))) @ random_orthogonal_matrix
    previous_chi = np.vstack((
                            np.hstack((random_orthogonal_matrix_2, v_init, p_init)),
                            np.hstack((np.zeros((2, 3)), np.eye(2)))
                        ))
    
    skew_5_5 = np.vstack((
        np.hstack((skew_np(state[0:3].reshape(3,)), v_imu, p_imu)),
        np.zeros((2, 5))
    ))
    
    R_imu = (expm(skew_5_5) @ previous_chi )[:3, :3]

    alpha2_phuc = (R_imu.T @ v_imu ).flatten() + np.cross(omega_n - b_omega, p_c)

    v_c = R_c.T @ alpha2_phuc

    omega_car = R_c.T @(omega_n - b_omega)
    vee_0 = (omega_n - b_omega).reshape(3,)
    omega_p = np.cross(vee_0, p_c)

    a_car = R_c.T @ (a_n - b_a + np.cross(vee_0, omega_p))

    ############## find analytical solution #################
    #  only print one time
    if(i == 0):
        ############ for check the derivation  ################
        M1 = np.array([0, 1, 0]).reshape(1, 3)
        M2 = np.array([1, 0, 0]).reshape(1, 3)
        M3 = np.array([0, 0, 1]).reshape(1, 3)

        alpha1 = (a_n - b_a + np.cross(vee_0, omega_p)).reshape(3,)
        # alpha2 = (R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)
        alpha2 = alpha2_phuc
        alpha3 = (vee_0).reshape(3,1)
        
        # for term 2/7 correct
        print("the 2/7 term : \n", - M2 @ (R_c.T @ R_imu.T) * (omega_car[2]))

        vee = (omega_n - b_omega).reshape(3,)
        pee = p_c.reshape(3,)

        term_4_7 =  M1 @ R_c.T @ ( - np.outer(vee, pee) - np.squeeze((omega_n - b_omega).dot(p_c)) * np.eye(3) + (2 * np.outer(pee, vee))) \
                    + (1 * M2 @ R_c.T @ skew_np(p_c)) * (-omega_car[2]) \
                    + (-1 * M3 @ R_c.T) * (-v_c[0])
        
        print("the 4/7 term : \n", term_4_7)

        #  for term 5/7 correct
        term_5_7 = -1 * M1 @ R_c.T 
        print("the 5/7 term : \n", term_5_7)

        #  for term 6/7 not correct
        term_6_7 = M1 @ R_c.T @ skew_np(alpha1) \
                   - np.squeeze(np.array([1,0,0]) @ R_c.T @ skew_np(alpha2)) * omega_car[2] \
                   - v_c[0] * (M3 @ R_c.T @ skew_np(alpha3))
        # print("pre 67", np.squeeze(np.array([1,0,0]) @ R_c.T @ skew_np(alpha2)))
        print("the 6/7 term : \n", term_6_7)

        #  for term 7/7 correct
        term_7_7 = M1 @ R_c.T @ skew_np(omega_n - b_omega) @ skew_np(omega_n - b_omega) \
                    + M2 @ R_c.T @ skew_np( - omega_n + b_omega) * (omega_car[2])
        print("the 7/7 term : \n", term_7_7)

    obs_new = a_car[1] - v_c[0] * omega_car[2]
    return  obs_new

def compute_jacobian_new_obser(x_0, a_n, omega_n, d=1e-9):
    # Define small perturbations
    n = len(x_0)
    jacobian = np.zeros((n))

    # Compute the observation at the nominal state
    obs_nominal = h_acc_new(x_0, a_n, omega_n, 2)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc_new(x_0_perturbed, a_n, omega_n, i)
        # Calculate Jacobian element by finite difference
        # print(obs_perturbed)
        jacobian[i] = (obs_perturbed - obs_nominal) / d

    return jacobian

# Example usage:
chi_imu = np.array([0.0, 0.0, 0.0])
v_imu = np.array([0.0, 0.0, 0.0])
p_imu = np.array([0.0, 0.0, 0.0])
b_a = np.array([0.0, 0.0, 0.0])
b_omega = np.array([0.5, 0.01, 0.01])
chi_c = np.array([0.0, 0.0, 0.0])
p_c = np.array([0.1, 0.1, 0.1])

# Nominal state

a_n = np.array([1.0, 1.0, 1.0])
omega_n = np.array([0.5, 0.1, 1.0])
x_0 = np.concatenate((chi_imu, v_imu, p_imu, b_omega, b_a, chi_c, p_c))

In [3]:
# compute_jacobian_paper(x_0, a_n, omega_n).T
compute_jacobian_new_obser(x_0, a_n, omega_n).reshape(21, 1)

the 2/7 term : 
 [[-0.29810314 -0.41313062 -0.08748245]]
the 4/7 term : 
 [[ 0.21765534 -0.01467591 -0.3337104 ]]
the 5/7 term : 
 [[0.75034456 0.63681192 0.17735169]]
the 6/7 term : 
 [[-0.39814419  0.6802402  -0.101711  ]]
the 7/7 term : 
 [[ 1.01829705  0.70419093 -0.06401736]]


array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.2981031 ],
       [-0.41313064],
       [-0.08748247],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.21765545],
       [-0.01467582],
       [-0.33371039],
       [ 0.75034445],
       [ 0.63681194],
       [ 0.17735169],
       [-0.39814396],
       [ 0.68024031],
       [-0.10171086],
       [ 1.01829722],
       [ 0.70419115],
       [-0.06401701]])

In [4]:
import numpy as np

alpha2_phuc = np.array([0.5, 1.0, 3.0])
epsilon_r0 = np.array([0.0, 0.0, 0.0])
def fx(epsilon_r):
    R_c = expm(skew_np(epsilon_r.reshape(3,))) @ random_orthogonal_matrix
    # alpha2_phuc = (R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)
    v_c = R_c.T @ alpha2_phuc
    jacobian_all = R_c.T @ skew_np(alpha2_phuc)
    print("analytical: ", np.array([1, 0, 0]).reshape(1, 3) @ jacobian_all)
    return v_c[0]


fx_0 = fx(epsilon_r0)
#  find jacobian w.r.t bias omega
jacobian = np.zeros(3)
for i in range(3):
    d = 1e-9
    # Compute the observation with perturbed state
    epsilon_r_cp = np.copy(epsilon_r0)
    epsilon_r_cp[i] +=  d
    fx_per = fx(epsilon_r_cp)
    jacobian[i] = (fx_per - fx_0) / d

print("numerical: ", jacobian)


analytical:  [[ 2.25314709  0.12851915 -0.41836423]]
analytical:  [[ 2.25314709  0.12851915 -0.41836423]]
analytical:  [[ 2.25314709  0.12851916 -0.41836423]]
analytical:  [[ 2.25314709  0.12851915 -0.41836423]]
numerical:  [ 2.253147    0.12851897 -0.41836445]
